In [54]:
!git add capstone.ipynb
!git commit -m "feat) 1. 간이 데이터 전처리 구현, 2. 사이킷런의 모델을 이용한 간단한 구현, 3. tensorflow를 이용한 간단한 모델 구현"
!git add gf_data.csv
!git commit -m "docs) 카페 할아버지 공장의 2023.05 1주차 "
!git push origin main

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.7/57.7 kB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 11.6 MB/s eta 0:00:00



In [1]:
pip install np_utils

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.0/62.0 kB 1.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for np_utils: filename=np_utils-0.6.0-py3-none-any.whl size=56441 sha256=b4ed53eb01f1ebad5ef1bc0c747975d88d18fc9f729b13a88aeebbc115b4aea3
  Stored in directory: /root/.cache/pip/wheels/b6/c7/50/2307607f44366dd021209f660045f8d51cb976514d30be7cc7
Successfully built np_utils


# import

In [2]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

import np_utils
import numpy
import tensorflow as tf
import matplotlib.pyplot as plt
import pandas as pd

print(os.getcwd())

/content


# 데이터 전처리
## 데이터 내용
- MONTH : 월
- DAY : 일
- DOW : 요일(Day of Week)
- TIME : 시간대

  시간대는 00\~06시, 06\~11시, 11\~14시, 14\~17시, 17\~21시, 21\~24시 로 나누어지며 각각을 처리하기 쉽도록 1부터 6가지의 INTEGER로 할당하였음.
- RAIN_PER : 강수량. (강수량 데이터 출처 : https://data.kma.go.kr/data/rmt/rmtList.do?code=420&pgmNo=572)
- SEARCH_VOL : 검색량.

  블랙키위(https://blackkiwi.net) 플랫폼에서 일별 검색량을 추출할 수 있음. 블랙키위의 일별 추출량과 Google 트랜드의 일일 검색량 추이 그래프를 통하여 시간대별 하루 검색량을 산출할 수 있음.
- PAYMENT : 매출

  소상공인시장진흥공단의 데이터 활용. 설정한 구역의 월별 매출, 요일별 평균 매출, 시간대별 평균 매출을 수집할 수 있음.

## 현재 시나리오
CSV파일에 5월 첫째주에 관련된 데이터들을 Raw하게 담아놓았음.

- MONTH, DAY, RAIN_PER, SEARCH_VOL은 그대로 모델에 학습.
- DOW는 1부터 7의 INTEGER를 이용하여 모델에 학습시킴.
- PAYMENT는 혼잡도를 직접적으로 나타낼 수 있는 과거의 데이터이므로 요일과 강수량, 검색량을 통하여 후보정한 뒤 우리가 예측해야할 값인 혼잡도(CONJESTION)를 결과값으로 모델에 학습시킴.

## Todo
5월 1주 데이터만 해도 42개의 row가 생김. 1년에 대해 데이터를 생성한다면 42*4*12개의 row를 직접 만드는 것은 불가능함.

#### 1. 강수량
기상청의 단기 예보 파일을 루트 디렉토리에 위치시킨 후 월, 일, 시간대를 읽어 코드 작성으로 데이터를 생성.

각 시간대별로 강수량을 나타내는 지표가 상당히 많음. 시간대에 맞게 강수량을 평균을 내어 데이터를 만들 것.

#### 2. 검색량
블랙키위의 일별 검색량을 바탕으로 Google 트랜드의 일별 검색량 추이를 이용하여 시간대 별 검색량을 산출해야함.

블랙키위의 일별 검색량을 직접 입력.
시간대별 그래프는 가장 검색량 추이를 잘 보여주는 그래프를 이용하여 각 요일에 대해 시간대별 검색량을 산출.

---
**ex**

일별 검색량 얻기.
- 방안 1
  블랙 키위에서 얻은 검색량 (특정 요일에 대한 검색량이 바로 나옴.)
  5/1(월) : 110 <br>
  5/2(화) : 120 <br>
  5/3(수) : 110 <br>
  5/4(목) : 120 <br>
  5/5(금) : 180 <br>
  5/6(토) : 210 <br>
  5/7(일) : 140 <br>

- 방안 2
  블랙 키위에서 얻은 5월 총 검색량 : 4270
  요일 별 검색량 비율
  월 : 13.6% <br>
  화 : 14.5% <br>
  수 : 13.2% <br>
  목 : 13.2% <br>
  금 : 14.1% <br>
  토 : 16.4% <br>
  일 : 15.0% <br>
  4270을 4주로 나눈 뒤 요일별 비율을 통하여 각 요일별 검색량 산출.

![gf_day_graph](/content/img/gf_day_graph.png)

시간대별 검색량 얻기

00-06 : 0 → 0% <br>
06-11 :73 → 27.13% <br>
11-14 : 100 → 37.17% <br>
14-17 : 47 → 17.47% <br>
17-21 : 49 → 18.21% <br>
21-24 : 0 → 0% <br>

---

#### 3. 결제량
월 매출을 4주로 나누어 월-일 비율에 맞추어 데이터를 산출 할 시 날씨에 관련된 지표가 반영이 안된 상태임.

날씨에 대한 영향을 후보정하기 위하여 강수량과 결제량에 따라 새로 지표를 생성할 필요가 있음. 강수량과 혼잡도에 대해 관련성을 이후 증명할 계획.

**ex.**

강수량

00-30 : 매출 * 1.2 <br>
31-50 : 매출 * 0.9 <br>
51-70 : 매출 * 0.75 <br>
71-90 : 매출 * 0.5 <br>
91~   : 매출 * 0.2 <br>




In [3]:
raw_data = pd.read_csv("./gf_data.csv")

print(raw_data.columns)
display(raw_data)

Index(['MONTH', 'DAY', 'DOW', 'TIME', 'RAIN_PER', 'SEARCH_VOL', 'PAYMENT'], dtype='object')


,MONTH,DAY,DOW,TIME,RAIN_PER,SEARCH_VOL,PAYMENT
0,5,1,MON,1,8.80,0.00,14.47
1,5,1,MON,2,12.83,157.41,57.88
2,5,1,MON,3,13.38,215.58,162.46
3,5,1,MON,4,13.97,101.32,236.79
4,5,1,MON,5,15.21,105.61,171.67
5,5,1,MON,6,16.64,0.00,14.47
6,5,2,TUE,1,17.83,0.00,16.88
7,5,2,TUE,2,34.58,167.97,67.52
8,5,2,TUE,3,34.91,230.13,189.54
9,5,2,TUE,4,35.27,108.16,276.25


In [4]:
def preprocess_dow(data):
  day_mapping = {'MON': 1, 'TUE': 2, 'WED': 3, 'THU': 4, 'FRI': 5, 'SAT': 6, 'SUN': 7}
  new_data = data.copy()
  new_data['DOW'] = new_data['DOW'].map(day_mapping)
  return new_data

def calculate_new_payment(data):
    new_data = data.copy()
    def multiplier(rain_per):
        if rain_per <= 30:
            return 1.2
        elif 31 <= rain_per <= 50:
            return 0.9
        elif 51 <= rain_per <= 70:
            return 0.75
        elif 71 <= rain_per <= 90:
            return 0.5
        elif 91 <= rain_per <= 100:
            return 0.2
        else:
            return 1
    new_data['NEW_PAYMENT'] = data.apply(lambda row: row['PAYMENT'] * multiplier(row['RAIN_PER']), axis=1)
    new_data.drop('PAYMENT', axis=1, inplace=True)
    return new_data

dow_processed_data = preprocess_dow(raw_data)
pmt_processed_data = calculate_new_payment(dow_processed_data)

x_train = pmt_processed_data.drop("NEW_PAYMENT", axis=1)
y_train = pmt_processed_data["NEW_PAYMENT"]

print("X_TRAIN_DATA")
print(x_train)
print("Y_TRAIN_DATA")
print(y_train)

y_train = y_train.to_numpy().astype('float32')
x_train = x_train.to_numpy().astype('float32')

X_TRAIN_DATA
    MONTH  DAY  DOW  TIME  RAIN_PER  SEARCH_VOL
0       5    1    1     1      8.80        0.00
1       5    1    1     2     12.83      157.41
2       5    1    1     3     13.38      215.58
3       5    1    1     4     13.97      101.32
4       5    1    1     5     15.21      105.61
5       5    1    1     6     16.64        0.00
6       5    2    2     1     17.83        0.00
7       5    2    2     2     34.58      167.97
8       5    2    2     3     34.91      230.13
9       5    2    2     4     35.27      108.16
10      5    2    2     5     38.48      112.74
11      5    2    2     6     42.18        0.00
12      5    3    3     1     45.48        0.00
13      5    3    3     2     39.58      152.91
14      5    3    3     3     39.98      209.50
15      5    3    3     4     40.86       98.46
16      5    3    3     5     44.66      102.63
17      5    3    3     6     49.32        0.00
18      5    4    4     1     50.64        0.00
19      5    4    4     2  

### scikit-learn의 제공 모델을 이용한 부분. (TEST 용)

In [36]:
X_train, X_test, Y_train, Y_test = train_test_split(x_train, y_train, test_size=0.2, random_state=42)
model = RandomForestRegressor(n_estimators=100, random_state=42)
model.fit(X_train, Y_train)
Y_pred = model.predict(X_test)
print(X_test)
print(Y_pred)
mse = mean_squared_error(Y_test, Y_pred)
rmse = numpy.sqrt(mse)

print("테스트 세트의 MSE: ", mse)
print("테스트 세트의 RMSE: ", rmse)
from sklearn.metrics import r2_score

# R^2 Score
r2 = r2_score(Y_test, Y_pred)
print(f"R^2 Score: {r2:.2f}")

# MAPE Calculation
def mean_absolute_percentage_error(y_true, y_pred):
    y_true, y_pred = numpy.array(y_true), numpy.array(y_pred)
    return numpy.mean(numpy.abs((y_true - y_pred) / y_true)) * 100

mape = mean_absolute_percentage_error(Y_test, Y_pred)
print(f"MAPE: {mape:.2f}%")


[[  5.     5.     5.     2.    22.56 163.34]
 [  5.     3.     3.     2.    39.58 152.91]
 [  5.     2.     2.     3.    34.91 230.13]
 [  5.     5.     5.     3.    21.69 223.78]
 [  5.     1.     1.     5.    15.21 105.61]
 [  5.     7.     7.     4.     0.29 111.89]
 [  5.     4.     4.     2.    38.78 152.91]
 [  5.     5.     5.     6.    12.71   0.  ]
 [  5.     6.     6.     1.    10.73   0.  ]]
[ 91.84112999  73.43943092 206.0716478  222.58229836 255.11745331
 267.27918327  75.62043053  22.29208056  19.4769405 ]
테스트 세트의 MSE:  779.4531514019533
테스트 세트의 RMSE:  27.918688210622527
R^2 Score: 0.92
MAPE: 21.57%


### V2 tensorflow를 이용한 모델 저장.

추후 오차값을 최소화하는 모델로 변경해야 함.

In [5]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, InputLayer
from tensorflow.keras.optimizers import Adam

model = Sequential([
    InputLayer(input_shape=(x_train.shape[1],)),
    Dense(64, activation='relu'),
    Dense(32, activation='relu'),
    Dense(1)
])

model.compile(optimizer=Adam(),
              loss='mse',
              metrics=['mae'])

model.summary()

history = model.fit(x_train, y_train, epochs=100, validation_split=0.2, batch_size=32)


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 64)                448       
                                                                 
 dense_1 (Dense)             (None, 32)                2080      
                                                                 
 dense_2 (Dense)             (None, 1)                 33        
                                                                 
Total params: 2561 (10.00 KB)
Trainable params: 2561 (10.00 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
Epoch 1/100
2/2 [==============================] - 1s 223ms/step - loss: 23048.6445 - mae: 116.0344 - val_loss: 41160.7930 - val_mae: 160.5596
Epoch 2/100
2/2 [==============================] - 0s 33ms/step - loss: 21489.0664 - mae: 111.0256 - val_loss: 39471.9883 - val_mae: 156.5550
E

### 모델 저장 부분

In [8]:
model.save("sales_prediction_model.h5")
print("모델 저장 완료.")

new_input = numpy.array([[5, 15, 3, 14, 40, 1200]])  # MONTH, DAY, DOW, TIME, RAIN_PER, SEARCH_VOL
new_input = new_input.astype('float32')
from tensorflow import keras

loaded_model = keras.models.load_model("sales_prediction_model.h5")

prediction = loaded_model.predict(new_input)
print(f"예측된 NEW_PAYMENT 값: {prediction[0][0]}")

모델 저장 완료.
1/1 [==============================] - 0s 99ms/step
예측된 NEW_PAYMENT 값: 962.1951293945312
